In [81]:
import sqlite3
import requests
import bs4

conn = sqlite3.connect('yahoo.db')
c = conn.cursor()

c.execute(''' create table if not exists yahoo_link(text TEXT, url TEXT, id INT)
    ''')

url = 'https://www.yahoo.com.tw'
re = requests.get(url)
soup = bs4.BeautifulSoup(re.content, 'lxml')

links = soup.find_all('a')
print(len(links))
for link in links:
    link_text =  link.text # 連結文字
    link_url = link.get('href')# 連結網址
    c.execute('insert into yahoo_link values(?, ?, ?)', (link_text, link_url, links.index(link)))

# 儲存資料庫
conn.commit()
conn.close()

139


## 作業

In [ ]:
import tkinter as tk
from datetime import datetime
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
import bs4
import sqlite3
import requests

import time
##################
def scrapy_thsrc():
    driver = webdriver.Chrome('')
    url = 'https://www.thsrc.com.tw'
    driver.get(url)
    print(driver.title)
    
    btn = driver.find_element(By.CLASS_NAME, 'swal2-confirm')
    btn.click()
    driver.maximize_window()

    # 尋找站名
    select = driver.find_element(By.ID ,'select_location01' )
    start = select.find_elements(By.TAG_NAME, 'option')
    start[convert_value1].click()
    print('起點站:', start[convert_value1].text)

    select2 = driver.find_element(By.ID ,'select_location02' )
    end = select2.find_elements(By.TAG_NAME, 'option')
    end[convert_value2].click()
    print('終點站:', end[convert_value2].text)
    # id = select_location01
    # id = select_location02
    time.sleep(1)

    search = driver.find_element(By.CLASS_NAME ,'btn-orange' )
    search.click()
    time.sleep(5)

    soup = bs4.BeautifulSoup(driver.page_source, 'lxml') # 透過pagesource連結 selenium 與 bs
    timeas = soup.find_all('span', class_ = 'font-16r darkgray')

    print('查詢時間最近的:', int(len(timeas)/2), '班車') # 看一下有幾個時間值

    now = datetime.now().strftime('%H:%M') # str format time
    print('現在時間是:', now)

    conn = sqlite3.connect('thsrc.db')
    c = conn.cursor()

    c.execute(''' create table if not exists mytable(出發 NUMERIC, 到達 NUMERIC)
        ''')

    query = f'''
            select * from mytable
            where 出發 > '{now}'
            order by 出發
            limit {x} '''
    
    for i in range(x):
        starttime =  timeas[i].text.strip() 
        arrivedtime = timeas[i+1].text.strip()
        c.execute('insert into mytable values(?, ?)', (starttime, arrivedtime))
    
    result = pd.read_sql_query(query, conn)

    # 儲存資料庫
    conn.commit()
    conn.close()
    print('你目前可以搭的車次是:\n', result)

In [ ]:
# 建立視窗
window = tk.Tk()
window.title('高鐵查詢程式')
global x
x = input('想查詢最近幾個班次?\n')

def check_digit(x):
    if x.isdigit() != True:
        print('請輸入1~5之間的整數')        
        return False  
    x = int(x)
    if x > 5 or x < 1:  
        print('最少1個班次, 最多5個班次')
        return False  
    return True  
while not check_digit(x):  
    x = input('想查詢最近幾個班次?\n')
    
x = eval(x)
font = ('微軟正黑體', 15)
# 建立起點查詢標籤
start_label = tk.Label(window, text = '起點站')
start_label.pack()

# 建立初始值標籤
# start_station_var = tk.IntVar()
start_station_var = tk.StringVar()
station_option = ['南港', '台北', '板橋', '桃園', '新竹',
                  '苗栗', '台中', '彰化', '雲林', '嘉義', '台南', '左營']
start_station_var.set(station_option[0]) # 設定初始值

def option_change1(event):
    global convert_value1
    convert_value1 = station_option.index(start_station_var.get())

def option_change2(event):
    global convert_value2
    convert_value2 = station_option.index(end_station_var.get())
    
# 下拉式選單
start_station_dropdown = tk.OptionMenu(window, start_station_var, *station_option, command = option_change1)

start_station_dropdown.config(width = 6)
start_station_dropdown.pack()

# 建立終點查詢標籤
end_label = tk.Label(window, text = '終點站')
end_label.pack()

# 建立初始值標籤
end_station_var = tk.StringVar()

end_station_var.set(station_option[0]) # 設定初始值

# 下拉式選單
end_station_dropdown = tk.OptionMenu(window, end_station_var, *station_option, command = option_change2)

end_station_dropdown.config(width = 6)
end_station_dropdown.pack()

scrapy_button = tk.Button(window, text = '開始查時刻表', command = scrapy_thsrc)
scrapy_button.pack()

result_text = tk.Text(window, font = font, width = 30)
result_text.pack()

window.mainloop()


想查詢最近幾個班次?
vd
請輸入1~5之間的整數
想查詢最近幾個班次?
5
台灣高鐵Taiwan High Speed Rail
起點站: 南港
終點站: 嘉義


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\ProgramData\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\USER\AppData\Local\Temp\ipykernel_4056\2195224286.py", line 42, in scrapy_thsrc
    if x > int(len(timeas)/2):
UnboundLocalError: local variable 'x' referenced before assignment
